#  **ICT303 - Assignment 2**

**Your name: Lee Ming Han**

**Student ID: 34307477**

**Email: chah3929@gmail.com**

In this assignment, you will build a deep learning model for identifying $120$ different breeds of dogs. Similar to the previous assignment, you will use real images from the [Kaggle competition](https://www.kaggle.com/c/dog-breed-identification).

In this assignment, your are required to use a ResNet network. You can use ResNet implementation provided in PyTorch. Note however  that there are many versions of ResNet (they differ in terms of number of layers). Your task is to find the best configuration that gives the best performance.

The rule is similar to the previous assignment:

1. Develop a better model to reduce the recognition error.  
2. Submit your results to Kaggle and take a sceenshot of your score. Then insert here the screenshot of your result.

It is important that you start as earlier as possible. Tuning hyper-parameters takes time, and Kaggle limits the number of submissions per day.

The top 3 students in the Kaggle ranking will be invited for a coffee!

In [1]:
# Import required libraries
import numpy as np
import pandas as pd
import cv2
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model, Model
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, BatchNormalization
from tensorflow.keras.applications.resnet_v2 import ResNet50V2, preprocess_input

In [2]:
# Read the CSV file and store it in the 'labels_file' variable
labels_file = pd.read_csv("labels.csv")

# Store training and testing images folder location
train_file = 'train/'
test_file = 'test/'

In [3]:
# Check the total number of unique dog breeds in the dataset
print("Unique Dog Breeds:", len(labels_file.breed.unique()))

Unique Dog Breeds: 120


In [4]:
# Specify the number of dog breeds in the dataset
num_breeds = 120

# Specify the batch size used during model training
batch_size = 64

# Create an instance of the LabelEncoder to encode categorical labels into numeric values
label_encoder = LabelEncoder()

In [5]:
# Get the breed_dict with breed counts
breed_dict = labels_file['breed'].value_counts().to_dict()

# Get the list of 120 unique breeds with highest counts
new_list = sorted(breed_dict.keys(), key=lambda x: breed_dict[x], reverse=True)[:num_breeds]

# Change the dataset to have those 120 unique breed records
labels_file = labels_file.query('breed in @new_list')

In [6]:
# Create a new column 'img_file' containing the image names with the image extension
labels_file['img_file'] = labels_file['id'].apply(lambda x: x + ".jpg")

In [7]:
labels_file

,id,breed,img_file
0,000bec180eb18c7604dcecc8fe0dba07,boston_bull,000bec180eb18c7604dcecc8fe0dba07.jpg
1,001513dfcb2ffafc82cccf4d8bbaba97,dingo,001513dfcb2ffafc82cccf4d8bbaba97.jpg
2,001cdf01b096e06d78e9e5112d419397,pekinese,001cdf01b096e06d78e9e5112d419397.jpg
3,00214f311d5d2247d5dfe4fe24b2303d,bluetick,00214f311d5d2247d5dfe4fe24b2303d.jpg
4,0021f9ceb3235effd7fcde7f7538ed62,golden_retriever,0021f9ceb3235effd7fcde7f7538ed62.jpg
...,...,...,...
10217,ffd25009d635cfd16e793503ac5edef0,borzoi,ffd25009d635cfd16e793503ac5edef0.jpg
10218,ffd3f636f7f379c51ba3648a9ff8254f,dandie_dinmont,ffd3f636f7f379c51ba3648a9ff8254f.jpg
10219,ffe2ca6c940cddfee68fa3cc6c63213f,airedale,ffe2ca6c940cddfee68fa3cc6c63213f.jpg
10220,ffe5f6d8e2bff356e9482a80a6e29aac,miniature_pinscher,ffe5f6d8e2bff356e9482a80a6e29aac.jpg


In [8]:
# Image size for resizing the images
image_size = 256

# Number of dataset records
num_records = len(labels_file)

# Initialize an array to store the preprocessed images
train_x = np.zeros((num_records, image_size, image_size, 3), dtype='float32')

# Iterate over the 'img_file' column of the DataFrame and preprocess the images
for i, img_id in enumerate(labels_file['img_file']):
    img = cv2.resize(cv2.imread('train/' + img_id, cv2.IMREAD_COLOR), (image_size, image_size))
    img_array = preprocess_input(np.expand_dims(np.array(img[..., ::-1].astype(np.float32)).copy(), axis=0))
    train_x[i] = img_array

In [9]:
# Initialize the LabelEncoder
encoder = LabelEncoder()

# Convert breed names into numerical format and store in 'train_y'
train_y = encoder.fit_transform(labels_file["breed"].values)

In [10]:
# Split the dataset into training and testing sets (80% training and 20% testing)
x_train, x_test, y_train, y_test = train_test_split(train_x, train_y, test_size=0.2, random_state=42)

In [11]:
# Image augmentation for the training set
train_datagen = ImageDataGenerator(rotation_range=45,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.25,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

# Generate augmented images for training sets
train_data_generator = train_datagen.flow(x_train,
                                     y_train,
                                     batch_size=batch_size)

# Image augmentation for the testing set
test_datagen = ImageDataGenerator()

# Generate augmented images for testing sets
test_data_generator = test_datagen.flow(x_test,
                                   y_test,
                                   batch_size=batch_size)

In [12]:
# Specify the number of dog breeds in the dataset
num_breeds = 120

# Build the ResNet50V2 model with input shape and pre-trained weights
resnet = ResNet50V2(input_shape=[image_size, image_size, 3], weights='imagenet', include_top=False)

# Freeze all trainable layers and train only top layers
for layer in resnet.layers:
    layer.trainable = False

# Add Global Average Pooling and Batch Normalization layers
x = resnet.output
x = BatchNormalization()(x)
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)

# Add a fully connected layer
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)

# Add the output layer with the number of dog breeds as the number of neurons and softmax activation
predictions = Dense(num_breeds, activation='softmax')(x)

# Create the final model with inputs from ResNet50V2 and outputs from the new output layer
model = Model(inputs=resnet.input, outputs=predictions)

In [13]:
#epochs for model training and learning rate for optimizer
epochs = 30
learning_rate = 1e-3

# Specify the batch size used during model training
batch_size = 64

#using RMSprop optimizer to compile or build the model
optimizer = RMSprop(learning_rate=learning_rate,rho=0.9)
model.compile(optimizer=optimizer,
              loss='sparse_categorical_crossentropy',
              metrics=["accuracy"])

#fit the training generator data and train the model
hist = model.fit(train_data_generator,
                 steps_per_epoch= x_train.shape[0] // batch_size,
                 epochs= epochs,
                 validation_data= test_data_generator,
                 validation_steps= x_test.shape[0] // batch_size)

#Save the model for prediction
model.save("model")

Epoch 1/30
127/127 [==============================] - 540s 4s/step - loss: 2.5931 - accuracy: 0.3869 - val_loss: 0.9292 - val_accuracy: 0.7303
Epoch 2/30
127/127 [==============================] - 513s 4s/step - loss: 1.4613 - accuracy: 0.5847 - val_loss: 0.7735 - val_accuracy: 0.7651
Epoch 3/30
127/127 [==============================] - 513s 4s/step - loss: 1.2899 - accuracy: 0.6279 - val_loss: 0.7452 - val_accuracy: 0.7692
Epoch 4/30
127/127 [==============================] - 508s 4s/step - loss: 1.2214 - accuracy: 0.6513 - val_loss: 0.7486 - val_accuracy: 0.7681
Epoch 5/30
127/127 [==============================] - 508s 4s/step - loss: 1.1571 - accuracy: 0.6672 - val_loss: 0.7535 - val_accuracy: 0.7747
Epoch 6/30
127/127 [==============================] - 508s 4s/step - loss: 1.0939 - accuracy: 0.6798 - val_loss: 0.7627 - val_accuracy: 0.7792
Epoch 7/30
127/127 [==============================] - 507s 4s/step - loss: 1.0677 - accuracy: 0.6874 - val_loss: 0.7745 - val_accuracy: 0.7767

INFO:tensorflow:Assets written to: model\assets


In [15]:
# Output the result to a csv file

import os

# Specify the batch size used during model training
batch_size = 64

# Assuming you have already loaded and trained the model
# and have the predictions in 'labels_file' DataFrame
model = load_model("model")

# Define the image size (assuming im_size is defined)
image_size = 256

# Path to the folder containing the test images
test_folder_path = 'test/'

# List to store predictions and corresponding image ids
predictions = []
image_ids = []

# List of unique breed names (assuming new_list is defined)
unique_breeds = sorted(new_list)

# Loop through the test images
for filename in os.listdir(test_folder_path):
    if filename.endswith(".jpg"):
        image_path = os.path.join(test_folder_path, filename)

        # Read and preprocess the image
        img_array = cv2.resize(cv2.imread(image_path, cv2.IMREAD_COLOR), (image_size, image_size))
        img_array = preprocess_input(np.expand_dims(np.array(img_array[..., ::-1].astype(np.float32)).copy(), axis=0))

        # Feed the model with the image array for prediction
        pred_val = model.predict(np.array(img_array, dtype="float32"))

        # Append the prediction probabilities to the predictions list
        predictions.append(pred_val[0])

        # Extract the image ID without the file extension
        image_id = os.path.splitext(filename)[0]
        image_ids.append(image_id)

# Create a DataFrame with the results
results_df = pd.DataFrame(predictions, columns=unique_breeds)
results_df.insert(0, 'id', image_ids)

# Set option to print the entire DataFrame with full precision
pd.set_option('display.float_format', lambda x: '%.10f' % x)

# Save the DataFrame to a CSV file
results_df.to_csv('submission.csv', index=False)



1/1 [==============================] - 0s 105ms/step


1/1 [==============================] - 0s 118ms/step


1/1 [==============================] - 0s 100ms/step


1/1 [==============================] - 0s 103ms/step


1/1 [==============================] - 0s 93ms/step


...

1/1 [==============================] - 1s 706ms/step


1/1 [==============================] - 1s 694ms/step


1/1 [==============================] - 1s 699ms/step


1/1 [==============================] - 1s 725ms/step


1/1 [==============================] - 1s 703ms/step


###### 

In [ ]:
# Kaggle result is 0.7902

import matplotlib.pyplot as plt

image_path = 'Kaggle-result.png'

# Read the image file using OpenCV
image = cv2.imread(image_path)

# Check if the image was successfully read
if image is not None:
    
    # Convert BGR to RGB format
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # Display the image using matplotlib
    plt.imshow(image_rgb)
    plt.axis('off')  # Hide the axis ticks and labels
    plt.show()
else:
    print("Failed to read the image.")

Hi sir, I apologize for the late submission. Actually, I had completed the assignment three days before the deadline, but I accidentally reran everything, which caused the previous output to disappear. In order to display all the output on the console, I had no choice but to wait for the whole model to run again. I am using a pretty old laptop to run everything on my CPU, as Google Colab kept crashing, which caused me to spend around three days to output all the results again. I am truly sorry, sir. I hope you won't penalize me for the late submission. Thank you.